##### Pacotes Necessários 

In [55]:
import psycopg2
import pandas as pd
from configparser import ConfigParser

##### Conexão com a Base e Extração de Dados

In [56]:
# Função para configuração de conexão com parâmetros de acesso presentes no arquivo database.ini
# Transforma o arquivo em uma variável de objeto
def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Seção {0} não encontrada no arquivo {1}'.format(section, filename))
    return db

In [57]:
def connect():
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cursor = conn.cursor()

        retorno = []

        cursor.execute('SELECT * FROM forms')
        forms = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'forms'")
        colunas = cursor.fetchall()
        retorno.append([forms, colunas])

        cursor.execute('SELECT * FROM orders')
        orders = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'orders'")
        colunas = cursor.fetchall()
        retorno.append([orders, colunas])

        cursor.execute('SELECT * FROM order_items')
        items = cursor.fetchall()
        cursor.execute("SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'order_items'")
        colunas = cursor.fetchall()
        retorno.append([items, colunas])

        cursor.close()
        return retorno
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Conexão encerrada.')


if __name__ == '__main__':
    retorno = connect()

Conexão encerrada.


In [58]:
forms = pd.DataFrame(retorno[0][0])
forms.columns = [item for sublist in retorno[0][1] for item in sublist]

orders = pd.DataFrame(retorno[1][0])
orders.columns = [item for sublist in retorno[1][1] for item in sublist]

order_items = pd.DataFrame(retorno[2][0])
order_items.columns = [item for sublist in retorno[2][1] for item in sublist]

##### Manipulação de dados

In [59]:
# Remoção de colunas
forms = forms.drop(['id_cliente','data','sistema_operacional','navegador','localizacao','locale','cidade','utm_source',
'caracteristica','tempo_procedimento'], axis = 1)

orders = orders[['id_pedido','target']]

order_items = order_items[['id_pedido','id_form']]

In [60]:
# Selecionando apenas os valores únicos da coluna 'id_form' da tabela 'order_items'
items_form = order_items.drop_duplicates(subset = ['id_form'])
# Left join da tabela 'forms' com a 'items_form' seguido de left join da tabela resultante com a tabela 'orders'. 
# df é a base de dados resultante
df = forms.merge(items_form, 
                 on = 'id_form', 
                 how = 'left').merge(orders, 
                                     on = 'id_pedido', 
                                     how = 'left').drop(['id_form','id_pedido'], axis = 1)

In [61]:
# Lidando com NAs
df.target.fillna(0, inplace = True)
df.dropna(axis = 0, inplace = True)
df.drop(df[df.efeitos_desejados == 'N/C'].index, axis = 0, inplace = True)
df.drop(df[df.procedimentos == 'N/C'].index, axis = 0, inplace = True)


In [62]:
# Transformação de variáveis
[df.estado, codes_estado] = pd.factorize(df.estado)
[df.tipo_cabelo, codes_tipo_cabelo] = pd.factorize(df.tipo_cabelo)
[df.comprimento, codes_comprimento] = pd.factorize(df.comprimento)
[df.tipo_fios, codes_tipo_fios] = pd.factorize(df.tipo_fios)
[df.dieta, codes_dieta] = pd.factorize(df.dieta)
[df.atividade_fisica, codes_atividade_fisica] = pd.factorize(df.atividade_fisica)
[df.frequencia_estresse, codes_frequencia_estresse] = pd.factorize(df.frequencia_estresse)
[df.faixa_etaria, codes_faixa_etaria] = pd.factorize(df.faixa_etaria)
[df.fragancia, codes_fragancia] = pd.factorize(df.fragancia)

In [63]:
df.head()

,estado,qtde_lavagem_semana,tipo_cabelo,comprimento,tipo_fios,procedimentos,dieta,atividade_fisica,frequencia_estresse,faixa_etaria,efeitos_desejados,fragancia,target
0,0,4,0,0,0,Nenhum,0,0,0,0,Efeito 7,0,1.0
1,0,7,1,1,1,Todos acima,1,1,0,1,Efeito 1,1,0.0
2,0,2,2,2,2,Todos acima,1,2,1,2,Efeito 7,0,0.0
3,1,5,2,2,2,Todos acima,0,2,1,0,Efeito 7,0,0.0
4,2,4,3,1,2,Tintura,0,3,0,3,Efeito 7,1,0.0


In [64]:
df['efeito1'] = 0
df['efeito2'] = 0
df['efeito3'] = 0
df['efeito4'] = 0
df['efeito5'] = 0
df['efeito6'] = 0
df['efeito7'] = 0
df['efeito8'] = 0
df['efeito10'] = 0

df.loc[df['efeitos_desejados'].str.contains('Efeito 1'), 'efeito1'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 2'), 'efeito2'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 3'), 'efeito3'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 4'), 'efeito4'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 5'), 'efeito5'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 6'), 'efeito6'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 7'), 'efeito7'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 8'), 'efeito8'] = 1
df.loc[df['efeitos_desejados'].str.contains('Efeito 10'), 'efeito10'] = 1

In [65]:
df['procedimentos1'] = 0
df['procedimentos2'] = 0
df['procedimentos3'] = 0
df['procedimentos4'] = 0
df['procedimentos5'] = 0

df.loc[df['procedimentos'].str.contains('Nenhum'), 'procedimentos1'] = 1
df.loc[df['procedimentos'].str.contains('Tintura'), 'procedimentos2'] = 1
df.loc[df['procedimentos'].str.contains('Corte agressivo'), 'procedimentos3'] = 1
df.loc[df['procedimentos'].str.contains('Exposição ao sol'), 'procedimentos4'] = 1
df.loc[df['procedimentos'].str.contains('Nenhum'), 'procedimentos5'] = 1

In [66]:
df.drop(['efeitos_desejados', 'procedimentos'], axis = 1, inplace=True)

In [67]:
df.head()

,estado,qtde_lavagem_semana,tipo_cabelo,comprimento,tipo_fios,dieta,atividade_fisica,frequencia_estresse,faixa_etaria,fragancia,...,efeito5,efeito6,efeito7,efeito8,efeito10,procedimentos1,procedimentos2,procedimentos3,procedimentos4,procedimentos5
0,0,4,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
1,0,7,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,2,2,2,2,1,2,1,2,0,...,0,0,1,0,0,0,0,0,0,0
3,1,5,2,2,2,0,2,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2,4,3,1,2,0,3,0,3,1,...,0,0,1,0,0,0,1,0,0,0


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426769 entries, 0 to 427949
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   estado               426769 non-null  int64  
 1   qtde_lavagem_semana  426769 non-null  int64  
 2   tipo_cabelo          426769 non-null  int64  
 3   comprimento          426769 non-null  int64  
 4   tipo_fios            426769 non-null  int64  
 5   dieta                426769 non-null  int64  
 6   atividade_fisica     426769 non-null  int64  
 7   frequencia_estresse  426769 non-null  int64  
 8   faixa_etaria         426769 non-null  int64  
 9   fragancia            426769 non-null  int64  
 10  target               426769 non-null  float64
 11  efeito1              426769 non-null  int64  
 12  efeito2              426769 non-null  int64  
 13  efeito3              426769 non-null  int64  
 14  efeito4              426769 non-null  int64  
 15  efeito5          

In [69]:
coluna = df.pop('target')
df = df.assign(target=coluna).astype('int64')